In [6]:
'''
!pip install pyyaml
!pip install matplotlib-venn
!pip install numpy
'''

'\n!pip install pyyaml\n!pip install matplotlib-venn\n!pip install numpy\n'

In [7]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
sns.set_style("whitegrid")
from pprint import pprint
import sqlite3
import yaml
import re
import datetime

try: # for pip >= 10
    from pip._internal.req import parse_requirements
except ImportError: # for pip <= 9.0.3
    from pip.req import parse_requirements

from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")

## Easier navigation:

------------------------------------

### 1. <a href='#setup'>Setup</a>
### 2. <a href='#merge'>Merging</a>
------------------------------------
### 3.  <a href='#specs'>Binder-Specs-EDA w/ clean_specs</a>

------------------------------------

<a name='setup'></a>
## 1. Setup

<br>

In [6]:
dash = '-' * 60

### *events Table*

In [14]:
con = sqlite3.connect('binder.sqlite')
df1 = pd.read_sql('SELECT * FROM events', con)

In [15]:
df1.columns

Index(['version', 'timestamp', 'provider', 'spec', 'origin', 'ref',
       'guessed_ref'],
      dtype='object')

In [16]:
df1

,version,timestamp,provider,spec,origin,ref,guessed_ref
0,1,2018-11-03T00:00:00+00:00,GitHub,Qiskit/qiskit-tutorial/master,None,None,27d67cc4485ebfebae2b36b4856f1cea9fc7f693
1,1,2018-11-03T00:00:00+00:00,GitHub,ipython/ipython-in-depth/master,None,None,7e5ce96cc9251083979efdfc393425f1229a4a68
2,1,2018-11-03T00:00:00+00:00,GitHub,QISKit/qiskit-tutorial/master,None,None,27d67cc4485ebfebae2b36b4856f1cea9fc7f693
3,1,2018-11-03T00:01:00+00:00,GitHub,QISKit/qiskit-tutorial/master,None,None,27d67cc4485ebfebae2b36b4856f1cea9fc7f693
4,1,2018-11-03T00:01:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,None,None,79184a07d47cf03787b39cfc345da98794d76554
...,...,...,...,...,...,...,...
18230449,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None
18230450,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None
18230451,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None
18230452,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None


### *spec_files Table*

In [40]:
con = sqlite3.connect('binder.sqlite')
df2 = pd.read_sql('SELECT * FROM spec_files', con)

In [41]:
df2.columns

Index(['ref', 'ls', 'runtime', 'apt', 'conda', 'pip', 'pipfile', 'julia', 'r',
       'nix', 'docker', 'setup', 'postbuild', 'start'],
      dtype='object')

In [42]:
df2

,ref,ls,runtime,apt,conda,pip,pipfile,julia,r,nix,docker,setup,postbuild,start
0,ab1be186a74c81c83f863bc3b6258fee46eeca56,"{"".ipynb_checkpoints"": {""I. Python and Jupyter...",None,None,None,numpy==1.18.5\npandas==1.0.5\nfuzzywuzzy==0.18...,None,None,None,None,None,None,None,None
1,9267f689d1935f5670e00d63b4f9639226f599ec,"{""test_script.R"": 115}",None,None,None,None,None,None,None,None,None,None,None,None
2,2fccf878bf9db2239207b39761d2fc9a78494610,"{"".ipynb_checkpoints"": {""binder-example-checkp...",None,None,name: binder-example\ndependencies:\n - pyt...,None,None,None,None,None,None,None,None,None
3,6c2b3723a3b7251a9b4df7d6fd9f0079b8df2ac1,"{"".Rbuildignore"": 14, "".binder"": {""Dockerfile""...",None,None,None,None,None,None,None,None,FROM rocker/binder:3.6.0\nLABEL maintainer='Mi...,None,None,None
4,6464ab1b75787590096275bf36b6e3035f23d2ab,"{"".gitignore"": 29, ""README.md"": 533, ""index.ip...",None,None,None,requests==2.24.0\nnumpy==1.19.2\nmatplotlib==3...,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178394,394381cccbbd63122129a09ca38775731901e1eb,"{"".gitignore"": 254, "".gitmodules"": 323, "".read...",None,None,None,None,None,None,None,None,None,None,b'#!/usr/bin/env bash\n\npython3 -m pip instal...,None
178395,64831e2cc32a4877c669b3d16156be7e2d25507d,"{"".gitignore"": 40, ""README.md"": 1982, ""SMBE202...",b'r-3.6-2021-05-26\n',b'libudunits2-dev\ngdal-bin\nlibgdal-dev\nlibg...,None,None,None,None,"b'install.packages(""devtools"")\ninstall.packag...",None,None,None,b'mkdir bin\n\n# installation instructions fro...,None
178396,2d5f216e18ac6ea8adc7f658d9bd344a49e7289c,"{"".gitignore"": 625, ""LICENSE"": 2615, ""README.m...",None,None,b'name: bestiapop\r\nchannels:\r\n - conda-fo...,None,None,None,None,None,None,b'import sys\r\nfrom setuptools import setup\r...,None,None
178397,95b20d80709832d340600461a86417ca5003bf7c,"{"".gitignore"": 1798, ""bear_classifier.ipynb"": ...",None,None,None,b'voila\nfastai>=2\npillow<7\npackaging\nipywi...,None,None,None,None,None,None,None,None


### *clean_specs Table*

In [37]:
con = sqlite3.connect('binder.sqlite')
df3 = pd.read_sql('SELECT * FROM clean_specs', con)

In [38]:
df3.columns

Index(['ref', 'conda_channels', 'conda_packages', 'pip_packages',
       'apt_packages'],
      dtype='object')

In [39]:
df3

,ref,conda_channels,conda_packages,pip_packages,apt_packages
0,ab1be186a74c81c83f863bc3b6258fee46eeca56,None,None,"[""numpy==1.18.5"", ""pandas==1.0.5"", ""fuzzywuzzy...",None
1,9267f689d1935f5670e00d63b4f9639226f599ec,None,None,None,None
2,2fccf878bf9db2239207b39761d2fc9a78494610,None,"[""python=3.7"", ""numpy"", ""pandas"", ""matplotlib""]",None,None
3,6c2b3723a3b7251a9b4df7d6fd9f0079b8df2ac1,None,None,None,None
4,6464ab1b75787590096275bf36b6e3035f23d2ab,None,None,"[""requests==2.24.0"", ""numpy==1.19.2"", ""matplot...",None
...,...,...,...,...,...
159641,394381cccbbd63122129a09ca38775731901e1eb,None,None,None,None
159642,64831e2cc32a4877c669b3d16156be7e2d25507d,None,None,None,"[""libudunits2-dev"", ""gdal-bin"", ""libgdal-dev"",..."
159643,2d5f216e18ac6ea8adc7f658d9bd344a49e7289c,"[""conda-forge"", ""defaults""]","[""python=3.7"", ""jinja2"", ""netcdf4"", ""tqdm"", ""r...","[""bestiapop""]",None
159644,95b20d80709832d340600461a86417ca5003bf7c,None,None,"[""voila"", ""fastai>=2"", ""pillow<7"", ""packaging""...",None


<a name='merge'></a>
## 2. Merging

<br>

Strategy: take the latest ref from `events` and merge that with the entries in `spec_files`. They have about the same amount of rows after `events` is filtered down to just its latest entries.

In [44]:
len(df1["ref"].unique())

201676

In [33]:
df1[df1["ref"].notnull()]

,version,timestamp,provider,spec,origin,ref,guessed_ref
9391434,4,2020-06-18T16:41:00+00:00,Git,https://bitbucket.org/fsmeraldi/jupytermarking...,gke.mybinder.org,d22a4b0a494b25ad9e0397b44d9abc3461eb6309,None
9391436,4,2020-06-18T16:41:00+00:00,GitHub,leosamu/pythonMoocCompanion/master,gke.mybinder.org,e0fe196804caf9a1dbd18ed2c045d3cb902c3c55,None
9391437,4,2020-06-18T16:41:00+00:00,GitHub,jupyterlab/jupyterlab-demo/try.jupyter.org,gke.mybinder.org,71b616307dcb9cba18262d8e5fa46ad828ca51e4,None
9391438,4,2020-06-18T16:41:00+00:00,GitHub,ipython/ipython-in-depth/master,gke.mybinder.org,7e5ce96cc9251083979efdfc393425f1229a4a68,None
9391439,4,2020-06-18T16:41:00+00:00,GitHub,SamLau95/nbinteract/master,gke.mybinder.org,502b0a8e5f6236ed23d4a3d0dcc22db72f6d31e4,None
...,...,...,...,...,...,...,...
18230449,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None
18230450,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None
18230451,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None
18230452,4,2021-06-06T23:58:00+00:00,GitHub,jupyterlab/jupyterlab-demo/master,gke.mybinder.org,5a5eb6bb04250b199a1cbb529e744075216a17a5,None


In [20]:
merged_df = pd.merge(df1, df2,  how='inner', on="ref")
merged_df

,version,timestamp,provider,spec,origin,ref,guessed_ref,ls,runtime,apt,conda,pip,pipfile,julia,r,nix,docker,setup,postbuild,start
0,4,2020-06-18T16:41:00+00:00,GitHub,leosamu/pythonMoocCompanion/master,gke.mybinder.org,e0fe196804caf9a1dbd18ed2c045d3cb902c3c55,None,"{"".gitignore"": 1879, ""LICENSE"": 1072, ""PythonU...",None,None,name: python-upvx\nchannels:\n - conda-forge\n,None,None,None,None,None,None,None,None,None
1,4,2020-06-18T16:42:00+00:00,GitHub,leosamu/pythonMoocCompanion/master,gke.mybinder.org,e0fe196804caf9a1dbd18ed2c045d3cb902c3c55,None,"{"".gitignore"": 1879, ""LICENSE"": 1072, ""PythonU...",None,None,name: python-upvx\nchannels:\n - conda-forge\n,None,None,None,None,None,None,None,None,None
2,4,2020-06-18T16:43:00+00:00,GitHub,leosamu/pythonMoocCompanion/master,gke.mybinder.org,e0fe196804caf9a1dbd18ed2c045d3cb902c3c55,None,"{"".gitignore"": 1879, ""LICENSE"": 1072, ""PythonU...",None,None,name: python-upvx\nchannels:\n - conda-forge\n,None,None,None,None,None,None,None,None,None
3,4,2020-06-18T16:44:00+00:00,GitHub,leosamu/pythonMoocCompanion/master,gke.mybinder.org,e0fe196804caf9a1dbd18ed2c045d3cb902c3c55,None,"{"".gitignore"": 1879, ""LICENSE"": 1072, ""PythonU...",None,None,name: python-upvx\nchannels:\n - conda-forge\n,None,None,None,None,None,None,None,None,None
4,4,2020-06-18T16:44:00+00:00,GitHub,leosamu/pythonMoocCompanion/master,gke.mybinder.org,e0fe196804caf9a1dbd18ed2c045d3cb902c3c55,None,"{"".gitignore"": 1879, ""LICENSE"": 1072, ""PythonU...",None,None,name: python-upvx\nchannels:\n - conda-forge\n,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8409705,4,2021-06-06T23:43:00+00:00,GitHub,bodkan/smbe2021/main,gke.mybinder.org,64831e2cc32a4877c669b3d16156be7e2d25507d,None,"{"".gitignore"": 40, ""README.md"": 1982, ""SMBE202...",b'r-3.6-2021-05-26\n',b'libudunits2-dev\ngdal-bin\nlibgdal-dev\nlibg...,None,None,None,None,"b'install.packages(""devtools"")\ninstall.packag...",None,None,None,b'mkdir bin\n\n# installation instructions fro...,None
8409706,4,2021-06-06T23:52:00+00:00,GitHub,JJguri/bestiapop/HEAD,ovh.mybinder.org,2d5f216e18ac6ea8adc7f658d9bd344a49e7289c,None,"{"".gitignore"": 625, ""LICENSE"": 2615, ""README.m...",None,None,b'name: bestiapop\r\nchannels:\r\n - conda-fo...,None,None,None,None,None,None,b'import sys\r\nfrom setuptools import setup\r...,None,None
8409707,4,2021-06-06T23:52:00+00:00,GitHub,mzhao8/bear_classifier/HEAD,ovh.mybinder.org,95b20d80709832d340600461a86417ca5003bf7c,None,"{"".gitignore"": 1798, ""bear_classifier.ipynb"": ...",None,None,None,b'voila\nfastai>=2\npillow<7\npackaging\nipywi...,None,None,None,None,None,None,None,None
8409708,4,2021-06-06T23:55:00+00:00,GitHub,mzhao8/bear_classifier/HEAD,ovh.mybinder.org,95b20d80709832d340600461a86417ca5003bf7c,None,"{"".gitignore"": 1798, ""bear_classifier.ipynb"": ...",None,None,None,b'voila\nfastai>=2\npillow<7\npackaging\nipywi...,None,None,None,None,None,None,None,None


<a name='specs'></a>
## 3. Binder-Specs-EDA w/ clean_specs

<br>